In [1]:
import pandas as pd
import requests 
import os
import time

In [2]:
# Read the csv file
df = pd.read_csv('memes_total.csv')

In [6]:
# store the images within the folder
folder = 'images'

# create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)

# loop through the images and download them
for index, row in df[7355:7356].iterrows():
    try:
        url = row['image_url']
        r = requests.get(url)
        if r.status_code == 200:
            with open(os.path.join(folder, url.split('/')[-1]), 'wb') as f:
                f.write(r.content)
                f.close()
                print("Downloaded - ", index)
    except:
        print("Error - ", index)
        # save the index of the image that failed to download
        with open('error.txt', 'a') as f:
            f.write(str(index) + str(row['id']) + str(row['image_url']) + "\n")
    # time.sleep(0.5)

Downloaded -  7355


In [8]:
# count the number of images in the folder
print(len(os.listdir(folder)))


8281


In [10]:
# check the error file
with open('error.txt', 'r') as f:
    print(f.read())

In [18]:
df['image_path'] = df['image_path'].str.replace('https://covid19hatefulmemes2.s3.us-east-2.amazonaws.com/img/', 'images/')

/tmp/ipykernel_168665/1589960623.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['image_path'] = df['image_path'].str.replace('https://covid19hatefulmemes2.s3.us-east-2.amazonaws.com/img/', 'images/')


In [23]:
# check if the images are downloaded
df['image_path'].apply(lambda x: os.path.exists(x)).value_counts()
# redownload the images that failed to download
df_failed = df[df['image_path'].apply(lambda x: os.path.exists(x)) == False]

In [28]:
df_failed.image_url.tolist()

['https://covid19hatefulmemes2.s3.us-east-2.amazonaws.com/img/EYtu1xLXgAA2t4T.jpg',
 'https://covid19hatefulmemes2.s3.us-east-2.amazonaws.com/img/FbS5iXYXkAAv_18.jpg',
 'https://covid19hatefulmemes2.s3.us-east-2.amazonaws.com/img/FdNM8atXEAMqMMm.jpg']

In [29]:
# remove the images that failed to download
df = df[df['image_path'].apply(lambda x: os.path.exists(x)) == True]

In [31]:
df.to_csv('memes_total.csv', index=False)

In [33]:
# split the data into train and test
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

df_train.to_csv('memes_train.csv', index=False)
df_test.to_csv('memes_test.csv', index=False)